<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Equations of General Relativistic Hydrodynamics (GRHD)

## Author: Patrick Nelson and Zach Etienne

## This notebook documents and constructs a number of quantities useful for building symbolic (SymPy) expressions for the equations of general relativistic force-free electrodynamics (GRFFE), using the same (Valencia) formalism as `IllinoisGRMHD`

**Notebook Status:** <font color='red'><b> In Progress </b></font>

**Validation Notes:** This tutorial notebook will be confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation). **Additional validation tests may have been performed, but are as yet, undocumented. (TODO)**

## Introduction

We write the equations of general relativistic hydrodynamics in conservative form as follows (adapted from Eqs. 41-44 of [Duez et al](https://arxiv.org/pdf/astro-ph/0503420.pdf)):

\begin{eqnarray}
\partial_t \tilde{S}_i &+& \partial_j \left(\alpha \sqrt{\gamma} T^j_{{\rm EM}i} \right) = \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu}_{\rm EM} g_{\mu\nu,i},
\end{eqnarray}
where we assume $T^{\mu\nu}_{\rm EM}$ is the electromagnetic stress-energy tensor:
$$
T^{\mu\nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu\nu} + b^\mu b^\nu,
$$
and 
$$
v^j = \frac{u^j}{u^0} \\
$$

Also we will write the 4-metric in terms of the ADM 3-metric, lapse, and shift using standard equations.

Thus the full set of input variables include:
* Spacetime quantities:
    * ADM quantities $\alpha$, $\beta^i$, $\gamma_{ij}$
* Hydrodynamical quantities:
    * 4-velocity $u^\mu$
* Electrodynamical quantities
    * Magnetic field $B^i$

### A Note on Notation

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.

For instance, in calculating the first term of $b^2 u^\mu u^\nu$, we use Greek indices:

```python
T4EMUU = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        # Term 1: b^2 u^{\mu} u^{\nu}
        T4EMUU[mu][nu] = smallb2*u4U[mu]*u4U[nu]
```

When we calculate $\beta_i = \gamma_{ij} \beta^j$, we use Latin indices:
```python
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += gammaDD[i][j] * betaU[j]
```

As a corollary, any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one: A Latin index in a four-vector will be incremented and a Greek index in a three-vector will be decremented (however, the latter case does not occur in this tutorial notebook). This can be seen when we handle $\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$:
```python
# \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} / 2
for i in range(DIM):
    for mu in range(4):
        for nu in range(4):
            S_tilde_rhsD[i] += alpsqrtgam * T4EMUU[mu][nu] * g4DDdD[mu][nu][i+1] / 2
```

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

Each family of quantities is constructed within a given function (**boldfaced** below). This notebook is organized as follows


1. [Step 1](#importmodules): Import needed NRPy+ & Python modules
1. [Step 2](#stressenergy): **compute_TEM4UU()**, **compute_TEM4UD()**: Define the stress-energy tensor $T^{\mu\nu}_{\rm EM}$ and $T^\mu_{{\rm EM}\nu}$
1. [Step 3](#primtoconserv): **compute_sqrtgammaDET()**, **compute_S_tildeD()**: Writing the conservative variables in terms of the primitive variables
1. [Step 4](#grffefluxes): Define the fluxes for the GRFFE equations
    1. [Step 4.a](#stildesourceterms): **compute_S_tilde_fluxUD()**: Define $\tilde{S}_i$ flux term for GRFFE equations
1. [Step 5](#grhdsourceterms): Define source terms on RHSs of GRFFE equations
    1. [Step 5.a](#stildeisourceterm): Define source term on RHS of $\tilde{S}_i$ equation
        1. [Step 5.a.i](#fourmetricderivs): **compute_g4DDdD()**: Compute $g_{\mu\nu,i}$ in terms of ADM quantities and their derivatives
        1. [Step 5.a.ii](#stildeisource): **compute_S_tilde_source_termD()**: Compute source term of the $\tilde{S}_i$ equation: $\frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i}$
1. [Step 6](#convertvtou): **u4U_in_terms_of_vU_apply_speed_limit()**: Conversion of $v^i$ to $u^\mu$ (Courtesy Patrick Nelson)
1. [Step 7](#declarevarsconstructgrffeeqs): Declare ADM and hydrodynamical input variables, and construct GRFFE equations
1. [Step 8](#code_validation): Code Validation against `GRFFE.equations` NRPy+ module
1. [Step 9](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='importmodules'></a>

# Step 1: Import needed NRPy+ & Python modules \[Back to [top](#toc)\]
$$\label{importmodules}$$

In [1]:
# Step 1: Import needed core NRPy+ modules
from outputC import *            # NRPy+: Core C code output module
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

<a id='stressenergy'></a>

# Step 2: Define the electromagnetic stress-energy tensor $T^{\mu\nu}_{\rm EM}$ and $T^\mu_{{\rm EM}\nu}$ \[Back to [top](#toc)\]
$$\label{stressenergy}$$

Recall from above that

$$
T^{\mu\nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{1}{2} b^2 g^{\mu\nu} + b^\mu b^\nu.
$$
Also 

$$
T^\mu_{{\rm EM}\nu} = T^{\mu\delta}_{\rm EM} g_{\delta \nu}
$$

In [2]:
# Step 2.a: Define T^{mu nu} (a 4-dimensional tensor)
def compute_TEM4UU(gammaDD,betaU,alpha, smallb4U, smallb2,u4U):
    global TEM4UU

    # Then define g^{mu nu} in terms of the ADM quantities:
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4UU_ito_BSSN_or_ADM("ADM",gammaDD,betaU,alpha)

    # Finally compute T^{mu nu}
    TEM4UU = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            TEM4UU[mu][nu] = smallb2*u4U[mu]*u4U[nu] \
                             + sp.Rational(1,2)*smallb2*AB4m.g4UU[mu][nu] \
                             + smallb4U[mu]*smallb4U[nu]

# Step 2.b: Define T^{mu}_{nu} (a 4-dimensional tensor)
def compute_TEM4UD(gammaDD,betaU,alpha, TEM4UU):
    global TEM4UD
    # Next compute T^mu_nu = T^{mu delta} g_{delta nu}, needed for S_tilde flux.
    # First we'll need g_{alpha nu} in terms of ADM quantities:
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD,betaU,alpha)
    TEM4UD = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            for delta in range(4):
                TEM4UD[mu][nu] += TEM4UU[mu][delta]*AB4m.g4DD[delta][nu]

<a id='primtoconserv'></a>

# Step 3: Writing the conservative variables in terms of the primitive variables \[Back to [top](#toc)\]
$$\label{primtoconserv}$$

Recall from above that the conservative variables may be written as 
\begin{align}
\tilde{S}_i  &= \alpha \sqrt{\gamma} T^0{}_i
\end{align}

$T^{\mu\nu}_{\rm EM}$ and $T^\mu_{{\rm EM}\nu}$ have already been defined $-$ all in terms of primitive variables. Thus we'll just need $\sqrt{\gamma}=$`gammaDET`, and all conservatives can then be written in terms of other defined quantities, which themselves are written in terms of primitive variables and the ADM metric.

In [3]:
def compute_sqrtgammaDET(gammaDD):
    global sqrtgammaDET
    gammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
    sqrtgammaDET = sp.sqrt(gammaDET)

def compute_S_tildeD(alpha, sqrtgammaDET, T4UD):
    global S_tildeD
    S_tildeD = ixp.zerorank1(DIM=3)
    for i in range(3):
        S_tildeD[i] = alpha*sqrtgammaDET*T4UD[0][i+1]

<a id='grffefluxes'></a>

# Step 4: Define the fluxes for the GRFFE equations \[Back to [top](#toc)\]
$$\label{grffefluxes}$$

<a id='stildesourceterms'></a>

## Step 4.a: Define $\tilde{S}_i$ flux term for GRFFE equations \[Back to [top](#toc)\]
$$\label{stildesourceterms}$$

Recall from above that
\begin{equation}
\partial_t \tilde{S}_i + \partial_j \left(\alpha \sqrt{\gamma} T^j{}_i \right) = \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i},
\end{equation}
where

Here we will define all terms that go inside the $\partial_j$'s on the left-hand side of the above equation:

In [4]:
# Step 4.d: S_tilde flux
def compute_S_tilde_fluxUD(gammaDD,betaU,alpha, sqrtgammaDET, T4UD):
    global S_tilde_fluxUD
    S_tilde_fluxUD = ixp.zerorank2(DIM=3)
    for j in range(3):
        for i in range(3):
            S_tilde_fluxUD[j][i] = alpha*sqrtgammaDET*T4UD[j+1][i+1]

<a id='grffesourceterms'></a>

# Step 5: Define source terms on RHSs of GRFFE equations \[Back to [top](#toc)\]
$$\label{grffesourceterms}$$

<a id='stildeisourceterm'></a>

## Step 5.a: Define source term on RHS of $\tilde{S}_i$ equation \[Back to [top](#toc)\]
$$\label{stildeisourceterm}$$

Recall from above
$$
\partial_t \tilde{S}_i + \partial_j \left(\alpha \sqrt{\gamma} T^j_{{\rm EM}i} \right) = \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu}_{\rm EM} g_{\mu\nu,i}.
$$
Our goal here will be to compute
$$
\frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu}_{\rm EM} g_{\mu\nu,i}.
$$

<a id='fourmetricderivs'></a>

### Step 5.a.i: Compute $g_{\mu\nu,i}$ in terms of ADM quantities and their derivatives \[Back to [top](#toc)\]
$$\label{fourmetricderivs}$$


To compute $g_{\mu\nu,i}$ we need to evaluate the first derivative of $g_{\mu\nu}$ in terms of ADM variables.

We are given $\gamma_{ij}$, $\alpha$, and $\beta^i$, and the 4-metric is given in terms of these quantities via
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$

Thus 
$$
g_{\mu\nu,k} = \begin{pmatrix} 
-2 \alpha\alpha_{,i} + \beta^j_{,k} \beta_j + \beta^j \beta_{j,k} & \beta_{i,k} \\
\beta_{j,k} & \gamma_{ij,k}
\end{pmatrix},
$$
where $\beta_{i} = \gamma_{ij} \beta^j$, so
$$
\beta_{i,k} = \gamma_{ij,k} \beta^j + \gamma_{ij} \beta^j_{,k}
$$

In [5]:
def compute_g4DDdD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD):
    global g4DDdD
    # Eq. 2.121 in B&S
    betaD = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            betaD[i] += gammaDD[i][j]*betaU[j]

    betaDdD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # Recall that betaD[i] = gammaDD[i][j]*betaU[j] (Eq. 2.121 in B&S)
                betaDdD[i][k] += gammaDD_dD[i][j][k]*betaU[j] + gammaDD[i][j]*betaU_dD[j][k]

    # Eq. 2.122 in B&S
    g4DDdD = ixp.zerorank3(DIM=4)
    for k in range(3):
        # Recall that g4DD[0][0] = -alpha^2 + betaU[j]*betaD[j]
        g4DDdD[0][0][k+1] += -2*alpha*alpha_dD[k]
        for j in range(3):
            g4DDdD[0][0][k+1] += betaU_dD[j][k]*betaD[j] + betaU[j]*betaDdD[j][k]

    for i in range(3):
        for k in range(3):
            # Recall that g4DD[i][0] = g4DD[0][i] = betaD[i]
            g4DDdD[i+1][0][k+1] = g4DDdD[0][i+1][k+1] = betaDdD[i][k]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # Recall that g4DD[i][j] = gammaDD[i][j]
                g4DDdD[i+1][j+1][k+1] = gammaDD_dD[i][j][k]

<a id='stildeisource'></a>

### Step 5.b.ii: Compute source term of the $\tilde{S}_i$ equation: $\frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i}$ \[Back to [top](#toc)\]
$$\label{stildeisource}$$

Now that we've computed `g4DDdD`$=g_{\mu\nu,i}$, the $\tilde{S}_i$ evolution equation source term may be quickly constructed.

In [6]:
# Step 5.b.ii: Compute S_tilde source term
def compute_S_tilde_source_termD(alpha, sqrtgammaDET,g4DDdD, T4UU):
    global S_tilde_source_termD
    S_tilde_source_termD = ixp.zerorank1(DIM=3)
    for i in range(3):
        for mu in range(4):
            for nu in range(4):
                S_tilde_source_termD[i] += sp.Rational(1,2)*alpha*sqrtgammaDET*T4UU[mu][nu]*g4DDdD[mu][nu][i+1]

<a id='convertvtou'></a>

# Step 6: Conversion of $v^i$ to $u^\mu$ (Courtesy Patrick Nelson) \[Back to [top](#toc)\]
$$\label{convertvtou}$$

This conversion is a bit subtle, as truncation error on $v^i$ may result in a superluminal or ultrarelativistic velocity, which in either case a GRHD code cannot handle reliably.

As described in the [`GiRaFFE` $\tilde{S}_i$ tutorial notebook](Tutorial-GiRaFFE_NRPy_Ccode_library-S_tilde-flux.ipynb#speed_limit), we have already devised an algorithm for computing $u^\mu$ from $v_{(n)}^i$, the Valencia 3-velocity, defined as (Eq 11 in the IllinoisGRMHD paper)

$$
v^i_{(n)} = \frac{1}{\alpha}\left(v^i + \beta^i\right).
$$

Thus we first construct the Valencia velocity from $v^i = u^i/u^0$, then call the function from `GiRaFFE` to apply when necessary the speed limit on the Lorentz factor `GAMMA_SPEED_LIMIT`, then output the limited $v^i_{(n)}$ and $u^0$.

Then from these quantities, $u^{i}$ is given by
\begin{align}
v^i_{(n)} &= \frac{1}{\alpha}\left(v^i + \beta^i\right) \\
v^i_{(n)} &= \frac{1}{\alpha}\left(\frac{u^i}{u^0} + \beta^i\right) \\
\implies \frac{u^i}{u^0} + \beta^i &= \alpha v^i_{(n)}\\
\implies u^i &= u^0 \left(\alpha v^i_{(n)} - \beta^i\right)
\end{align}

In [7]:
# Step 6: Convert v^i into u^\mu, applying a speed limiter
def u4U_in_terms_of_vU_apply_speed_limit(alpha,betaU,gammaDD, vU):
    global u4_ito_3velsU
    
    import GiRaFFE_HO.Stilde_flux as GSf
    ValenciavU = ixp.zerorank1()
    for i in range(3):
        ValenciavU[i] = (vU[i] + betaU[i])/alpha

    GSf.compute_u0_noif(gammaDD,alpha,ValenciavU)
    u4_ito_3velsU = ixp.zerorank1(DIM=4)
    u4_ito_3velsU[0] = GSf.rescaledu0
    for i in range(3):
        u4_ito_3velsU[i+1] = GSf.rescaledu0 * (alpha * GSf.rescaledValenciavU[i] - betaU[i])

<a id='declarevarsconstructgrffeeqs'></a>

# Step 7: Declare ADM and hydrodynamical input variables, and construct GRFFE equations \[Back to [top](#toc)\]
$$\label{declarevarsconstructgrffeeqs}$$

In [8]:
# First define hydrodynamical quantities
u4U = ixp.declarerank1("u4U", DIM=4)
smallb4U = ixp.declarerank1("smallb4U", DIM=4)
smallb2 = sp.symbols('smallb2',real=True)

# Then ADM quantities
gammaDD = ixp.declarerank2("gammaDD","sym01",DIM=3)
betaU   = ixp.declarerank1("betaU", DIM=3)
alpha   = sp.symbols('alpha', real=True)

# First compute stress-energy tensor T4UU and T4UD:
compute_TEM4UU(gammaDD,betaU,alpha, smallb4U, smallb2,u4U)
compute_TEM4UD(gammaDD,betaU,alpha, TEM4UU)

# Next sqrt(gamma)
compute_sqrtgammaDET(gammaDD)

# Compute conservative variables in terms of primitive variables
compute_S_tildeD( alpha, sqrtgammaDET, TEM4UD)

# Next compute fluxes of conservative variables
compute_S_tilde_fluxUD(gammaDD,betaU,alpha, sqrtgammaDET,    TEM4UD)

# Then declare derivatives & compute g4DDdD
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01",DIM=3)
betaU_dD   = ixp.declarerank2("betaU_dD"  ,"nosym",DIM=3)
alpha_dD   = ixp.declarerank1("alpha_dD"          ,DIM=3)
compute_g4DDdD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD)

# Then compute source terms on tau_tilde and S_tilde equations
compute_S_tilde_source_termD(alpha, sqrtgammaDET, g4DDdD, TEM4UU)

# Finally compute the 4-velocities in terms of an input 3-velocity testvU
testvU = ixp.declarerank1("testvU",DIM=3)
u4U_in_terms_of_vU_apply_speed_limit(alpha,betaU,gammaDD, testvU)

<a id='code_validation'></a>

# Step 8: Code Validation against `GRFFE.equations` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$

As a code validation check, we verify agreement in the SymPy expressions for the GRHD equations generated in
1. this tutorial verus
2. the NRPy+ [GRHD.equations](../edit/GRHD/equations.py) module.

In [9]:
import GRFFE.equations as Ge

# First compute stress-energy tensor T4UU and T4UD:
Ge.compute_TEM4UU(gammaDD,betaU,alpha, smallb4U, smallb2,u4U)
Ge.compute_TEM4UD(gammaDD,betaU,alpha, Ge.TEM4UU)

# Next sqrt(gamma)
Ge.compute_sqrtgammaDET(gammaDD)

# Compute conservative variables in terms of primitive variables
Ge.compute_S_tildeD( alpha, Ge.sqrtgammaDET, Ge.TEM4UD)

# Next compute fluxes of conservative variables
Ge.compute_S_tilde_fluxUD (gammaDD,betaU,alpha,Ge.sqrtgammaDET,Ge.TEM4UD)

# Then declare derivatives & compute g4DDdD
# gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01",DIM=3)
# betaU_dD   = ixp.declarerank2("betaU_dD"  ,"nosym",DIM=3)
# alpha_dD   = ixp.declarerank1("alpha_dD"          ,DIM=3)
Ge.compute_g4DDdD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD)

# Finally compute source terms on tau_tilde and S_tilde equations
Ge.compute_S_tilde_source_termD(    alpha, Ge.sqrtgammaDET,Ge.g4DDdD,Ge.TEM4UU)

GetestvU = ixp.declarerank1("testvU")
Ge.u4U_in_terms_of_vU_apply_speed_limit(alpha,betaU,gammaDD, GetestvU)

In [10]:
all_passed=True
def comp_func(expr1,expr2,basename,prefixname2="Ge."):
    if str(expr1-expr2)!="0":
        print(basename+" - "+prefixname2+basename+" = "+ str(expr1-expr2))
        all_passed=False

def gfnm(basename,idx1,idx2=None,idx3=None):
    if idx2==None:
        return basename+"["+str(idx1)+"]"
    if idx3==None:
        return basename+"["+str(idx1)+"]["+str(idx2)+"]"
    return basename+"["+str(idx1)+"]["+str(idx2)+"]["+str(idx3)+"]"

expr_list = []
exprcheck_list = []
namecheck_list = []

namecheck_list.extend(["sqrtgammaDET"])
exprcheck_list.extend([Ge.sqrtgammaDET])
expr_list.extend([sqrtgammaDET])
for mu in range(4):
    namecheck_list.extend([gfnm("u4_ito_3velsU",mu)])
    exprcheck_list.extend([Ge.u4_ito_3velsU[mu]])
    expr_list.extend([u4_ito_3velsU[mu]])    
    for nu in range(4):
        namecheck_list.extend([gfnm("TEM4UU",mu,nu),gfnm("TEM4UD",mu,nu)])
        exprcheck_list.extend([Ge.TEM4UU[mu][nu],Ge.TEM4UD[mu][nu]])
        expr_list.extend([TEM4UU[mu][nu],TEM4UD[mu][nu]])
        for delta in range(4):
            namecheck_list.extend([gfnm("g4DDdD",mu,nu,delta)])
            exprcheck_list.extend([Ge.g4DDdD[mu][nu][delta]])
            expr_list.extend([g4DDdD[mu][nu][delta]])
            
        
for i in range(3):
    namecheck_list.extend([gfnm("S_tildeD",i),gfnm("S_tilde_source_termD",i)])
    exprcheck_list.extend([Ge.S_tildeD[i],Ge.S_tilde_source_termD[i]])
    expr_list.extend([S_tildeD[i],S_tilde_source_termD[i]])
    for j in range(3):
        namecheck_list.extend([gfnm("S_tilde_fluxUD",i,j)])
        exprcheck_list.extend([Ge.S_tilde_fluxUD[i][j]])
        expr_list.extend([S_tilde_fluxUD[i][j]])

for i in range(len(expr_list)):
    comp_func(expr_list[i],exprcheck_list[i],namecheck_list[i])

if all_passed:
    print("ALL TESTS PASSED!")

ALL TESTS PASSED!


<a id='latex_pdf_output'></a>

# Step 9: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GRHD_Equations-Cartesian-new.pdf](Tutorial-GRHD_Equations-Cartesian-new.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GRFFE_Equations-Cartesian-new.ipynb
!pdflatex -interaction=batchmode Tutorial-GRFFE_Equations-Cartesian-new.tex
!pdflatex -interaction=batchmode Tutorial-GRFFE_Equations-Cartesian-new.tex
!pdflatex -interaction=batchmode Tutorial-GRFFE_Equations-Cartesian-new.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GRFFE_Equations-Cartesian-new.ipynb to latex
[NbConvertApp] Writing 66648 bytes to Tutorial-GRFFE_Equations-Cartesian-new.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
